In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np 
import os,sys
from PIL import Image

In [ ]:
def grayscale(picture):
    res= Image.new(picture.mode, picture.size)
    width, height = picture.size

    for i in range(0, width):
        for j in range(0, height):
            pixel=picture.getpixel((i,j))
            avg=int((pixel[0]+pixel[1]+pixel[2])/3)
            res.putpixel((i,j),(avg,avg,avg))
    res.show()
    return res

In [ ]:
def normalize(picture):
    width, height = picture.size
    normalized_array = []
    for j in range(0, height):
	    for i in range(0, width):
		    pixel = picture.getpixel((i,j))
		    normalized_array.append( pixel[0] / 255.0 )
    return np.array(normalized_array)

In [ ]:
def loadDmgLoc():
  image_size = 150
  #Load the VGG model
  vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

  # Freeze the layers except the last 4 layers
  for layer in vgg_conv.layers[:-4]:
      layer.trainable = False

  # Create the model
  model = models.Sequential()
  
  # Add the vgg convolutional base model
  model.add(vgg_conv)
  
  # Add new layers
  model.add(layers.Flatten())
  model.add(layers.Dense(1024, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(3, activation='softmax'))
  
  # Show a summary of the model. Check the number of trainable parameters
  model.summary()

  model.load_weights('/content/drive/My Drive/dataset_zapsure/dmgloc_weights.h5')
  return model

def dmgLoc(imgpath,model):
  image_size = 150
  img = Image.open(imgpath).resize((image_size,image_size))
  img_arr = np.expand_dims(img_to_array(img), axis=0)

  image = preprocess_input(img_arr)
  prediction = model.predict(image)

  maxval = prediction.max()
  if(maxval == prediction[0][0]):
    return 0 #bonnet damage
  elif(maxval == prediction[0][1]):
    return 1 #sides damage
  else:
    return 2 #rear damage